## Clone repository

In [1]:
!git clone https://github.com/lamnguyenvu98/RWF2000-pytorch-version.git

Cloning into 'RWF2000-pytorch-version'...
remote: Enumerating objects: 1679, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 1679 (delta 11), reused 30 (delta 7), pack-reused 1635
Receiving objects: 100% (1679/1679), 37.37 MiB | 36.97 MiB/s, done.
Resolving deltas: 100% (410/410), done.


## Mount training data

Dataset Structure:

```
RWF2000-Small-Build (root)
    ├── train
    |    ├── Fight
    |    |     ├── data1.npz
    |    |     ├── data2.npz
    |    |     └── ...
    |    └── NonFight
    |          ├── data1.npz
    |          ├── data2.npz
    |          └── ...
    |
    └── val
         ├── Fight
         |     ├── data1.npz
         |     ├── data2.npz
         |     └── ...
         └── NonFight
               ├── data1.npz
               ├── data2.npz
               └── ...
```

In [ ]:
!python /content/drive/MyDrive/mount_cloud.py pcloud --local /content/pcloud

In [11]:
%%capture
!unzip /content/pcloud/Datasets/ZipDataset/RWF2000-Small-Build.zip -d /content

## Create log and checkpoint directory

In [4]:
!mkdir -p /content/log_folder
!mkdir -p /content/log_folder/checkpoints

## Create yaml file

In [14]:
%%writefile /content/rwf2000.yaml
TRAIN:
  BATCH_SIZE: 8
  NUM_FRAMES: 64
  ACCUMULATE_BATCH: 1
  EPOCHS: 30
  LEARNING_RATE: 1.0e-2
  MOMENTUM: 9.0e-1
  WEIGHT_DECAY: 1.0e-6
SCHEDULER:
  STEP_SIZE: 3
  GAMMA: 7.0e-1
VALIDATION:
  SAVE_LAST: True
  TOP_K: 3
SETTINGS:
  ACCELERATOR: "gpu"
  DEVICES: 1
  PRECISION: 16-mixed
  RESUME: False
NEPTUNE_LOGGER:
  API_TOKEN:
  PROJECT:
  WITH_ID:
  TAGS:
DIR:
  RESUME_CHECKPOINT: '/content/log_folder/checkpoints/last.ckpt'
  CHECKPOINT_DIR: '/content/log_folder/checkpoints/'
  DATA_DIR: '/content/RWF2000-Small-Build'
  LOG_DIR: '/content/log_folder'

Overwriting /content/rwf2000.yaml


## Install missing packages on Colab

In [6]:
!pip install lightning --quiet
!pip install torchmetrics --quiet
!pip install neptune --quiet
!pip install dotmap --quiet
!pip install "ray[serve]" --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.4/722.4 kB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Change dir to repo

In [7]:
%cd /content/RWF2000-pytorch-version

/content/RWF2000-pytorch-version


## Create pyproject.toml for Colab

This file only for colab, remove installing dependencies step since Colab already has most of them.

In [8]:
%%writefile /content/RWF2000-pytorch-version/pyproject.toml
[build-system]
requires = ["setuptools>=61.0", "setuptools-scm"]
build-backend = "setuptools.build_meta"

[project]
name = "FlowGatedNetwork"
version = "1.0.0"
authors = [
    {name = "Vu Lam Nguyen", email = "lamnguyenvu98@gmail.com"},
]
description = "Unofficial Implementation of Flow Gated Network. This network was trained on RWF2000 dataset"
requires-python = "==3.10.*"

[tool.setuptools]
packages=["src"]
include-package-data=true

[project.scripts]
export-jit = "src.export_jit:main"

Overwriting /content/RWF2000-pytorch-version/pyproject.toml


## Install packages

In [ ]:
!python -m pip install -e .

## Train model

In [ ]:
!python examples/train.py --config /content/rwf2000.yaml